In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights
import json

In [54]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),       
    transforms.ToTensor(),                 
    transforms.Normalize(                 
        mean=[0.485, 0.456, 0.406],        
        std=[0.229, 0.224, 0.225]
    )
])



train_dataset = ImageFolder(root="Test", transform=transform)
test_dataset = ImageFolder(root="Training", transform=transform)

In [55]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

In [57]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)

In [58]:
num_classes = len(train_dataset.classes) 

model.fc = nn.Linear(model.fc.in_features, num_classes)

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [62]:
from torch.utils.data import Subset

train_dataset_small = Subset(train_dataset, list(range(1000)))
train_loader = DataLoader(train_dataset_small, batch_size=64, shuffle=True, num_workers=8)


In [ ]:
with open('class_to_idx.json', 'w') as f:
    json.dump(train_dataset.class_to_idx, f)

In [ ]:
num_epochs = 10 

for epoch in range(num_epochs):
    model.train() 
    rloss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        rloss += loss.item()

    avg_loss = rloss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct / total

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
torch.save(model, 'model.pth')